# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****eJgA
LANGSMITH_API_KEY=****2eed
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ject


In [2]:
from langchain.agents import create_agent

In [3]:
agent = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a full-stack comedian",
)

## No Streaming (invoke)

In [4]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

I’m a full‑stack comedian: my frontend jokes get laughs, my backend jokes return a 500. 

Want another?


## values
You have seen this streaming mode in our examples so far. 

In [5]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why did the scarecrow win an award? Because he was outstanding in his field.

Want another one?


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [6]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

We pack our socks and giggles, head out the door,
With shoes that insist they were made to explore.
Grandma brings stories that taste like warm toast,
While Dad claims he spotted a “giraffe” on the coast.

We race down the sidewalk—slow-motion, not fast,
Because winning’s a bonus; the fun is what lasts.
A squirrel lends advice on how to share snacks,
(Translation: “Hide kale under cookies,” — he didn’t come back.)

We build a small fort from blankets and cheer,
Invite the whole neighborhood: everyone’s near.
The sun naps a little behind the tall trees,
As crickets rehearse tiny symphonies on breeze.

At bedtime we whisper the day into dreams,
Of moonbeams that sparkle in river-like streams.
Home is a chorus, both clumsy and kind —
The kind of loud love that you’d gladly rewind.

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [7]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='d1805e2e-45ec-4b8a-8bdb-9bcaf8ac6b1f')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='d1805e2e-45ec-4b8a-8bdb-9bcaf8ac6b1f'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 132, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbKjfgtFUhDtJdjPIpAPAneTTqso', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--8ae6c6bf-2e11-4b30-b308-fe23a6ea3ed3-0', tool_calls=[{'name': 'get_we

In [8]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco')
('custom', 'Acquired data for city: San Francisco')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])